**0.Import Dependencies**

In [ ]:
import os
from PIL import Image
import numpy as np
import pickle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.notebook import tqdm

from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model

from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception


from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.activations import relu,softmax
from keras.initializers import he_normal,glorot_normal,random_normal,glorot_uniform
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta,Adam
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')

**1.Get the Zip file of data and extract it in the current session**

In [ ]:
! wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
! unzip '/content/nature_12K.zip'

**2. Common Class For creating CNN Architecture,Training and Predicting.**

In [ ]:
class Fine_Tune:

  #target_size = (w x h x channel)
  def __init__(self,target_size, num_neurons,num_hlayers,pretrained_model,trainable_layers):
    self.model = Sequential()
    self.target_size = target_size
    self.num_neurons = num_neurons
    self.num_hlayers = num_hlayers
    self.pretrained_model = pretrained_model
    self.trainable_layers = trainable_layers

  def BUILD_MODEL(self):
    if (self.pretrained_model == 'InceptionV3'):
      prt = InceptionV3(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 31
      
    elif (self.pretrained_model == 'InceptionResnetV2'):
      prt = InceptionResNetV2(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 36

    elif (self.pretrained_model == 'ResNet50'):
      prt = ResNet50(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 34

    elif (self.pretrained_model == 'Xception') :
      prt = Xception(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 17
    else:
      raise Exception("this model not used!!")
   
    if self.trainable_layers == 'No':
      for layer in prt.layers:
          layer.trainable = False
    else:
      for layer in prt.layers[:-1*trl]:
          layer.trainable = False

    self.model.add(prt)

    #Add dense layers.. 
    self.model.add(Flatten())

    for i in range(self.num_hlayers):
      self.model.add(Dense(units = self.num_neurons))
      self.model.add(Activation('relu'))

    self.model.add(Dense(units=10))
    self.model.add(Activation('softmax'))

    self.model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return


  def Fit_Model(self,train_path,epochs,batch_size):
    tr_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    horizontal_flip=True,
                                    validation_split = 0.2)    #0.2 data used as a validation data..

    tr_gen = tr_datagen.flow_from_directory(
            directory = train_path,target_size = self.target_size[:2],color_mode = 'rgb',       
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 10,
            subset = 'training'
            )

    val_gen = tr_datagen.flow_from_directory(
            directory = train_path,target_size = self.target_size[:2],color_mode = 'rgb',
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 10,
            subset = 'validation'
            ) 
   
    step_size_train = tr_gen.n//tr_gen.batch_size
    step_size_valid = val_gen.n//val_gen.batch_size

    self.model.fit_generator(generator = tr_gen,
                             steps_per_epoch = step_size_train,
                             validation_data = val_gen,
                             validation_steps = step_size_valid,
                             epochs = epochs,
                             verbose = 1)
    return

  def Predict_Test(self,test_path):

    ts_datagen = ImageDataGenerator(rescale = 1./255)
    test_gen = ts_datagen.flow_from_directory(
        directory = test_path,
        target_size = self.target_size[:2],
        color_mode = 'rgb',
        batch_size = 1,
        class_mode = None,
        shuffle = False,
        seed=10
        )
    
    step_size_test = test_gen.n//test_gen.batch_size
    test_gen.reset()
    print('Getting predictions for test data..')
    pred = self.model.predict_generator(generator = test_gen,
                                  steps = step_size_test,
                                  verbose = 1
                                  )
    ytrue = test_gen.classes ; ypred = np.argmax(pred,axis=1)
    return np.sum(ytrue == ypred)/len(ytrue)

**3.Train the model on the training data**

In [ ]:
train_path = '/content/inaturalist_12K/train'

target_size = (256,256,3)

obj1 = Fine_Tune(target_size=target_size ,
                   pretrained_model = 'ResNet50',          #select from ['InceptionV3' ,'InceptionResnetV2' , 'ResNet50' , 'Xception' ]
                   num_neurons =64,                        #select any number
                   num_hlayers = 1,                        #select from [1,2,3]
                   trainable_layers='No')                  #select from ['Yes','No']
obj1.BUILD_MODEL()
obj1.Fit_Model(train_path,epochs=1,batch_size=32)

94773248/94765736 [==============================] - 1s 0us/step
Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.
250/250 [==============================] - 269s 936ms/step - loss: 4.9386 - accuracy: 0.1147 - val_loss: 2.3027 - val_accuracy: 0.0998


**4.Get the accuracy for the test data**

In [ ]:
test_path = '/content/inaturalist_12K/val'
obj1.Predict_Test(test_path)